## Logisk imputering
<font size = 2>Logisk imputering er å tilordne en variabel en verdi utledet fra logiske regler:
- Hvis betingelse er sann, så skal verdi A settes
- Hvis  betingelse er sann, så skal verdi A settes, hvis ikke, så skal verdi B settes

### Funksjoner for logisk imputering
<font size=2>Det er blitt laget tre funksjonene i Python som henholdsvis endrer en negativ verdi til tallet 0, en negativ verdi til positiv verdi, og en missingverdi til tallet 0. Selve koden finnes i neste paragraf og eksempler på hvordan de brukes finnes lenger ned i noten:
<li> <code>set_neg_to_zero(df, col_names)</code>:
<li> <code>set_neg_to_plus(df, col_names)</code>:
<li> <code>set_miss_to_zero(df, col_names)</code>:
<ul>
</font>

In [ ]:
def set_neg_to_zero(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: 0 if x<0 else x)
    return df

def set_neg_to_plus(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: abs(x) if x<0 else x)
    return df
    
def set_miss_to_zero(df, col_names):
    for col_name in col_names:
        df[col_name] = df[col_name].apply(lambda x: 0 if pd.isna(x) else x)
    return df

#### Importere biblioteker

In [ ]:
import dapla as dp
import pandas as pd

#### Leser inn eksempeldataene

In [ ]:
df_areal = dp.read_pandas("/felles/veiledning/pyspark/eksempler/areal")
df_bnp = dp.read_pandas("/felles/veiledning/pyspark/eksempler/bnp")
df_innbyggerantall_2020 = dp.read_pandas("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
df_innbyggerantall_2018 = dp.read_pandas("/felles/veiledning/pyspark/eksempler/innbyggerantall/2018")

#### Koble innbyggerantall på arealdatasett (koblingsnøkkel er Landkode) til bruk for å illustrere logisk imputering (kode)

In [ ]:
df_areal_innbyggerantall_20 = pd.merge(df_areal, df_innbyggerantall_2020, how='left', on='Landkode')
df_areal_innbyggerantall_20

#### Erstatte missing-verdier i innbyggertall ved bruk av funksjonen set_miss_to_zero (kode)

In [ ]:
df_areal_innbyggerantall_20 = set_miss_to_zero(df_areal_innbyggerantall_20,col_names={'Innbyggerantall','År','Kilde_y','Land_y'})

In [ ]:
df_areal_innbyggerantall_20